# Unit 2 Assignment: Building a Mixture of Experts (MoE) Router

**Topic:** Advanced Architecture using Groq API  
**Estimated Time:** 45-60 Minutes  
**Tools:** Python, Groq API, Dotenv

---

## 🎯 Objective
Your task is to build a **"Smart Customer Support Router"** using a Mixture of Experts (MoE) architecture.

In a real-world company, you don't want a "Generalist" AI handling everything. You want:
1.  A **Technical Expert** for bug reports.
2.  A **Billing Expert** for refund requests.
3.  A **Sales Expert** for new inquiries.

You will build a **Router** that takes a user query, decides which expert is best suited for it, and then forwards the query to that specific expert configuration.

---

## 🛠️ Requirements

### 1. Setup
- Install `groq` and `python-dotenv`.
- Set up your Groq API Key (variable: `GROQ_API_KEY`).

### 2. Define Your Experts
Create a configuration dictionary `MODEL_CONFIG` where you define your experts.
*Note: Since we are using the Groq API, we can simulate different "experts" by using different **System Prompts** while using the same base model (e.g., `mixtral-8x7b-32768`).*

- **Technical Expert:** System prompt should be rigorous, code-focused, and precise.
- **Billing Expert:** System prompt should be empathetic, financial-focused, and policy-driven.
- **General Expert:** A fallback for casual chat.

### 3. The Router (The Core Task)
Write a function `route_prompt(user_input)` that uses an LLM call to classify the intent.
- Input: User's query string.
- Output: The **Category Name** (e.g., "technical", "billing", "general").
- **Constraint:** The router must return *only* the category name, nothing else.

### 4. The Orchestrator
Write a main function `process_request(user_input)` that:
1.  Calls `route_prompt` to decide the category.
2.  Selects the correct **System Prompt** based on the category.
3.  Calls the generic LLM (Mixtral) with that specific System Prompt + User Input.
4.  Returns the final answer.

---

## 📝 Example Output

**Input:**
> "My python script is throwing an IndexError on line 5."

**Logic:**
1.  **Router** sees "Python", "IndexError" -> Classifies as **"technical"**.
2.  **Orchestrator** loads the "Technical Expert" system prompt.
3.  **Expert** responds with a code fix.

**Input:**
> "I was charged twice for my subscription this month."

**Logic:**
1.  **Router** sees "charged", "subscription" -> Classifies as **"billing"**.
2.  **Orchestrator** loads the "Billing Expert" system prompt.
3.  **Expert** responds with refund policy info.

---

## 💡 Hints
- Use `temperature=0` for the Router (you want consistency).
- Use `temperature=0.7` for the Experts (you want creativity/flexibility).
- Your routing prompt should be very clear: *"Classify this text into one of these categories: [technical, billing, general]. Return ONLY the word."*

---

## 🚀 Bonus Challenge
Can you add a **"Tool Use"** expert?
If the user asks for "current price of Bitcoin", route it to a function that (mock) fetches data, instead of just an LLM response.

---
    
## **Implementation below**
SRN: PES2UG23CS578  
Section: 6-I

In [2]:
!pip install groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.6 MB/s eta 0:00:00


In [3]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your GROQ API key: ")

Enter your GROQ API key: ··········


In [4]:
import os
from groq import Groq

client = Groq(api_key=os.environ["GROQ_API_KEY"])

BASE_MODEL = "llama-3.3-70b-versatile"


# Model configs
MODEL_CONFIG = {
    "technical": "You are a Technical Support Expert. Provide precise, logical, code-focused debugging help. Explain errors clearly and include fixes. ",

    "billing": "You are a Billing Support Expert. Respond empathetically and help resolve billing, refunds, and payments. ",

    "general": "You are a helpful general customer support assistant."
}


# Router
def route_prompt(user_input):
    router_prompt = f"""
Classify this customer request into ONE category:

technical
billing
general

Return ONLY the category name.

Request: {user_input}
"""

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a classifier."},
            {"role": "user", "content": router_prompt}
        ]
    )

    category = response.choices[0].message.content.strip().lower()

    if category not in MODEL_CONFIG:
        category = "general"

    return category


# =========================
# Expert caller
# =========================

def call_expert(category, user_input):

    config = MODEL_CONFIG[category]

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0.7,
        messages=[
            {"role": "system", "content": config},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content


# =========================
# Orchestrator
# =========================

def process_request(user_input):

    category = route_prompt(user_input)

    print("\nRouter selected:", category)

    response = call_expert(category, user_input)

    return response

In [6]:
queries = [

    "My python script throws an error on this line int a = (float) 86;",

    "I was charged twice for my subscription",

    "Hello, how are you?"
]

for q in queries:

    print("\n\nUser:", q)

    result = process_request(q)

    print("\nExpert:", result)
    print("\n")
    print("="*50)



User: My python script throws an error on this line int a = (float) 86;

Router selected: technical

Expert: # Step-by-step analysis of the problem:
1. **Syntax Error**: The line `int a = (float) 86;` contains a syntax error. Python does not use the same syntax for type casting as languages like C or Java.
2. **Invalid Type Casting**: In Python, you don't need to explicitly cast a value to a specific type. Python can automatically handle the conversion between types.
3. **Incorrect Variable Declaration**: Python does not require an explicit type declaration for variables. Instead, you can simply assign a value to a variable.

# Fixed solution:
```python
a = int(86.0)  # or simply a = 86
```

# Explanation of changes:
* **Removed explicit type declaration**: Python does not require explicit type declarations for variables.
* **Used Python's type conversion functions**: If you need to convert a value to a specific type, use Python's built-in functions like `int()`, `float()`, `str()`, 